libs


In [1]:
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import matplotlib.pyplot as plt

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abelc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


inisiasi leksikon


In [2]:
# import
PREPROCESSED_DATASET_FILE_NAME = "manual_preprocessing.csv"
PREPROCESSED_DATASET_FILE_PATH = f"{PREPROCESSED_DATASET_FILE_NAME}"
DATA_FRAME_PREPROCESSED_DATASET = pd.read_csv(PREPROCESSED_DATASET_FILE_PATH)
DATA_FRAME_PREPROCESSED_DATASET_LENGTH = len(DATA_FRAME_PREPROCESSED_DATASET)
print(f"preprocessed dataset: {DATA_FRAME_PREPROCESSED_DATASET_LENGTH} tweets")
display(DATA_FRAME_PREPROCESSED_DATASET.tail(1))

preprocessed dataset: 4 tweets


,text,cleaned_text,after_slang_text,underscore_negation_text,swap_negation_text,final_negation_text,after_stopwords_text,after_stemming_text,preprocessed_text
3,@TirtoID Ditawarin sosok yg cuma bikin taman a...,ditawarin sosok yg cuma bikin taman aja ada an...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok bikin taman analisanya memikir...,ditawarin sosok bikin taman analisanya pikir b...,ditawarin sosok bikin taman analisanya pikir b...


In [4]:
with open("../lexicons/lexicon_json_inset-neg.txt") as f:
  inset_neg = f.read()
with open("../lexicons/lexicon_json_inset-pos.txt") as f:
  inset_pos = f.read()

insetNeg = json.loads(inset_neg)
insetPos = json.loads(inset_pos)

In [5]:
# inisiasi vader sentiment analyzer
sia_inset_neg = SentimentIntensityAnalyzer()
sia_inset_pos = SentimentIntensityAnalyzer()

# clear default lexicon
sia_inset_neg.lexicon.clear()
sia_inset_pos.lexicon.clear()

# assign InSet lexicon
sia_inset_neg.lexicon.update(insetNeg)
sia_inset_pos.lexicon.update(insetPos)

labelling


In [6]:
# polarity score
def get_inset_compound_score(text):
  inset_compound_score = sia_inset_neg.polarity_scores(text)["compound"] + sia_inset_pos.polarity_scores(text)["compound"]
  return inset_compound_score

DATA_FRAME_PREPROCESSED_DATASET["compound_score"] = DATA_FRAME_PREPROCESSED_DATASET["preprocessed_text"].apply(get_inset_compound_score)
display(DATA_FRAME_PREPROCESSED_DATASET.tail(1))

,text,cleaned_text,after_slang_text,underscore_negation_text,swap_negation_text,final_negation_text,after_stopwords_text,after_stemming_text,preprocessed_text,compound_score
3,@TirtoID Ditawarin sosok yg cuma bikin taman a...,ditawarin sosok yg cuma bikin taman aja ada an...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok bikin taman analisanya memikir...,ditawarin sosok bikin taman analisanya pikir b...,ditawarin sosok bikin taman analisanya pikir b...,-0.4598


In [7]:
# labelling
def get_inset_label(number):
  if number < 0:
    return "negatif"
  elif number > 0:
    return "positif"
  else:
    return "netral"

DATA_FRAME_PREPROCESSED_DATASET["sentiment_label"] = DATA_FRAME_PREPROCESSED_DATASET["compound_score"].apply(get_inset_label)
display(DATA_FRAME_PREPROCESSED_DATASET.tail(1))

,text,cleaned_text,after_slang_text,underscore_negation_text,swap_negation_text,final_negation_text,after_stopwords_text,after_stemming_text,preprocessed_text,compound_score,sentiment_label
3,@TirtoID Ditawarin sosok yg cuma bikin taman a...,ditawarin sosok yg cuma bikin taman aja ada an...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok yang cuma bikin taman saja ada...,ditawarin sosok bikin taman analisanya memikir...,ditawarin sosok bikin taman analisanya pikir b...,ditawarin sosok bikin taman analisanya pikir b...,-0.4598,negatif


export


In [8]:
DATA_FRAME_PREPROCESSED_DATASET.to_csv("manual_sentiment.csv", index=False)